Pipelines are a utility component of several sub-toolkits in this package.

A Pipeline consists of a source, which generates values, any number of modifiers, which modify values, and terminal, which uses the values. The helper puztool.P has a variety of useful modifiers and terminals ready for use:

In [1]:
from puztool import P

words = ['gemini', 'jodhpurs', 'balderdash', 'facetiously']

words | P.filter(lambda x: 'a' in x) | P.all()

['balderdash', 'facetiously']

It's also easy to write your own functions with some provided decorators:

In [2]:
import itertools, funcy as fy
x = range(4)
a, b, c = itertools.tee(x, 3)
fy.interleave(a|P, b, c) | P.all()

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]

In [17]:
import itertools
from puztool.text import as_a
from puztool.pipeline import source, modifier, item_mod, terminal

# @source turns any generator function into a function that returns a pipeline
@source
def strings(n):
    '''Generate candidate  strings from numbers that are letter indices possibly minus 13'''
    if not n:
        yield ''
    else:
        for rest in strings(n[1:]):
            yield as_a(n[0])+rest
            yield as_a(n[0]+12)+rest

@modifier
def vowels_alphabetical(seq):
    '''Filter items that don't have vowels in alphabetical order.'''
    for x in seq:
        vowels = ''.join(c for c in x if c in 'aeiouy')
        if vowels == ''.join(sorted(vowels)):
            yield x

@modifier
def reverse(seq):
    for x in seq:
        yield x[::-1]

@terminal
def show(seq):
    print("Results:")
    print('\n'.join(f'  {x}' for x in seq))

strings((1,2,3,5,4)) | vowels_alphabetical() | reverse() | show()

Results:
  decba
  decbm
  decna
  decnm
  dqcba
  dqcbm
  dqcna
  dqcnm
  dqoba
  dqobm
  dqona
  dqonm
  pecba
  pecbm
  pecna
  pecnm
  pqcba
  pqcbm
  pqcna
  pqcnm
  pqoba
  pqobm
  pqona
  pqonm


Usefully, any iterable piped into a pipeline will be treated as a source of its contents, and any function piped to or from a pipeline will be treated as a modifier that updates one item at a time:

In [2]:
['gemini', 'jodhpurs', 'balderdash', 'facetiously'] | vowels_alphabetical() | (lambda x: x[1:-1]) | show()

Results:
  emin
  odhpur
  acetiousl


Of course, you can't pipe a plain list to a plain function and expect pipeline magic to happen, so the helper puztool.P is a no-op pipeline that helps with this:

In [3]:
from puztool import P

['foo', 'bar', 'baz'] | P | (lambda x:x[1:]) | show()

Results:
  oo
  ar
  az


P also has a variety of helpers for common modifiers and terminals:

In [4]:
['a', 'b', 'c', 'd'] | P.limit(2) | P.df()

,0
0,a
1,b


In [8]:
[list('xyz'), (1,2,3), 'abc'] | P.df(columns=['a', 'b', 'c'])

,a,b,c
0,x,y,z
1,x,y,z
2,1,2,3
3,a,b,c


In [1]:
from puztool import P, pipeline
import funcy as fy

range(10) | P.limit(8) | str | P.chunks(2, ' ') | P.split() | P.all()

[Result(val='0', provenance=(('0 1', 0),)),
 Result(val='1', provenance=(('0 1', 1),)),
 Result(val='2', provenance=(('2 3', 0),)),
 Result(val='3', provenance=(('2 3', 1),)),
 Result(val='4', provenance=(('4 5', 0),)),
 Result(val='5', provenance=(('4 5', 1),)),
 Result(val='6', provenance=(('6 7', 0),)),
 Result(val='7', provenance=(('6 7', 1),))]

In [18]:
import funcy as fy
fy.chunks?

Signature: fy.chunks(n, step, seq=EMPTY)
Docstring:
Lazily chunks seq into parts of length n or less.
Skips step items between parts if passed.
File:      ~/.local/share/virtualenvs/Tools-SytqyKAH/lib/python3.7/site-packages/funcy/seqs.py
Type:      function


In [2]:
'foo'.join('bar')

'bfooafoor'

In [13]:
import funcy as fy
fy.take?

Signature: fy.take(n, seq)
Docstring:
Returns a list of first n items in the sequence,
or all items if there are fewer than n.
File:      ~/.local/share/virtualenvs/Tools-SytqyKAH/lib/python3.7/site-packages/funcy/seqs.py
Type:      function


In [1]:
import z3
from puztool import pt, lowers
data = pt('''
back 76
drum 57
fine 40
grew 185
july 57
okay 200
pear 135
sigh 140
stop 350
vein 80
wild 112
zinc 26
''')
vars = {c:z3.Int(c) for c in lowers.strip() if c not in 'qx'}

In [2]:
cons = []
for v in vars.values():
    cons.append(z3.Or(v==1, v==5, v==10, v==25, v==50, v==100))
for _, word, val in data.itertuples():
    a,b,c,d = word
    cons.append(vars[a]+vars[b]+vars[c]+vars[d] == val)
from puztool.logic.solve import solve, all_solns
a, b = all_solns(cons)